In [1]:
import qiskit
from qiskit import transpile, ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit_aer import Aer
#from qiskit import IBMQ
from configparser import RawConfigParser

In [2]:
type = 'real' # Run program on the simulator or real quantum machine.
type = 'sim'

def run(program, type, shots = 1024):
  if type == 'real':
    if not run.isInit:
        # Setup the API key for the real quantum computer.
        parser = RawConfigParser()
        parser.read('config.ini')
        IBMQ.enable_account(parser.get('IBM', 'key'))
        run.isInit = True

    # Set the backend server.
    backend = qiskit.providers.ibmq.least_busy(qiskit.IBMQ.backends(simulator=False))

    # Execute the program on the quantum machine.
    print("Running on", backend.name())
    job = qiskit.execute(program, backend)
    return job.result().get_counts()
  else:
    # Execute the program in the simulator.
    print("Running on the simulator.")
    simulator = Aer.get_backend('qasm_simulator')
    transpiled_program = transpile(program, simulator)
    job = simulator.run(transpiled_program, shots=shots)
    return job.result().get_counts()

run.isInit = False

## Hello World

In [3]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
program = QuantumCircuit(qr, cr);

In [5]:
program.measure(qr, cr)
print(run(program, type))

Running on the simulator.
{'00': 1024}


## superposition

In [6]:
# setupqubits.
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
program = QuantumCircuit(qr, cr);

# superpositie van eerste qubit
program.h(qr[0]) # hadamard gate
# entangle subit 1 en qubit 2
program.cx(qr[0], qr[1])
# Meet de qubits
program.measure(qr, cr);

# Execute the program.
print(run(program, type))

Running on the simulator.
{'00': 535, '11': 489}


# een programma om informatie te versturen van Zender naar Ontvanger

In [10]:
# Setup qubits.
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
program = QuantumCircuit(qr, cr);

# Zender: breng qubit 0 in superpositie
program.h(qr[0])
# Zender: entangle qubit 0 en qubit 1
program.cx(qr[0], qr[1])

## intermezzo: geef de 2e qubit aan de ontvanger. Momenteel is er nog geen informatie op het kanaal doorgegeven.
## we willen nu aan de ontvanger de boodschap (1) doorgaven. Hiervoor willen we zorgen dat nadat de 

# Zender : Invert the first qubit to set it from 0 to 1 (we willen 11 krijgen maar manipeleren enkel de eerste qubit vanaf nu q[0]).
# 00  I  - Identity nothing to do
# 10  X  - program.x(qr[0])
# 01  Z  - program.z(qr[0])
# 11  XZ - program.x(qr[0]) program.z(qr[0])
program.z(qr[0]) # Z is the 'minus' operator: 0 maps to 0, 1 to -1
program.x(qr[0]) # X is the invert operator, or the NOT gate

# Zender: undo entanglement
program.cx(qr[0], qr[1])
# Zender: undo superposition
program.h(qr[0])


# Ontvanger: Measure the value of the qubits, we krijgen de originele waardes terug
program.measure(qr, cr);

# Execute the program.
print(run(program, type))

Running on the simulator.
{'11': 1024}


# Superdense coding

In [ ]:
# een programma 
# Setup qubits.
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
program = QuantumCircuit(qr, cr);

# Sender: superpoition
program.h(qr[0])
# Sender: entanglement
program.cx(qr[0], qr[1])

# Sender: Invert the first qubit to set it from 0 to 1
# 00  I  - Identity nothing to do
# 10  X  - program.x(qr[0])
# 01  Z  - program.z(qr[0])
# 11  XZ - program.x(qr[0]) program.z(qr[0])
program.z(qr[0])

# Receiver: Repeat the controlled NOT operator, reversing the entanglement.
program.cx(qr[0], qr[1])

# Receiver: Repeat the Hadamard, reversing the superposition state.
program.h(qr[0])

# Receiver: Measure the value of the qubits, we should get back the original values.
program.measure(qr, cr);

# Execute the program.
print(run(program, type))

Running on the simulator.
{'01': 1024}
